## 6. Calculations for comparison paper

## Setting things up
You need to install VolFe once on your machine, if you haven't yet. Then we need to import a few Python packages (including VolFe).

In [1]:
# Install VolFe on your machine. Don't remove the # from this line!
# pip install VolFe # Remove the first # in this line if you have not installed VolFe on your machine before.

# import python packages
import pandas as pd
import matplotlib.pyplot as plt
import VolFe as vf

### Kīlauea_Dryend ###

The low H2O content of this one means it can struggle to solve the equations at times. This means it requires a little trial-and-error to work out what pressure step size, solve step size, and solve species to use to get it to work, as you'll see below...

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Kīlauea_Dryend',
           'T_C': 1220., # Temperature in 'C
           'SiO2': 50.142, # wt%
           'TiO2': 2.257, # wt%
           'Al2O3': 12.224, # wt%
           'FeOT': 11.33, # wt%
           'MnO': 0.323, # wt%
           'MgO': 10.215, # wt%
           'CaO': 10.476, # wt%
           'Na2O': 1.754, # wt%
           'K2O': 0.398, # wt%
           'P2O5': 0.233, # wt%
           'H2O': 0.3, # wt%
           'CO2ppm': 800., # ppm
           'STppm': 1500., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.18}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

In [3]:
# Choosing model options so that the print status works - just helps to see where the calculation is getting to
my_models = [['print status',"True"]]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# running calculation
degas1 = vf.calc_gassing(my_analysis,models=my_models)

Kīlauea_Dryend
T= 1220.0 P= 1711.9992252609893 2024-07-15 14:57:14.341987
1700.0 : Switching solve species from OCS to OCH (first time)
1120.0 : Switching solve species from OCH to OHS (first time)
1120.0 : Switching solve species from OHS to OCS (second time)
solver failed, calculation aborted at P =  1120.0 2024-07-15 14:57:35.166965


/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/equilibrium_equations.py:2504: RuntimeWarning: invalid value encountered in scalar power
  xm_H2O_ = (K4_*y_H2O_*xg_H2O_*P)**0.5


It failed to solve at P = 1120. 

So now we extract the mole fractions of the gas species that VolFe uses to solve the equations from the last successful pressure step (1130 bar) as initial guesses to set it off again and choose the P to start calculations off at.

In [4]:
my_analysis["xg_O2"] = degas1.loc[degas1['P_bar']==1130.0,'xgO2_mf'] # mole fraction of O2 from the last sucessful P step of 1130
my_analysis["xg_H2"] = degas1.loc[degas1['P_bar']==1130.0,'xgH2_mf'] # mole fraction of H2 from the last sucessful P step of 1130
my_analysis["xg_S2"] = degas1.loc[degas1['P_bar']==1130.0,'xgS2_mf'] # mole fraction of S2 from the last sucessful P step of 1130
my_analysis["xg_CO"] = degas1.loc[degas1['P_bar']==1130.0,'xgCO_mf'] # mole fraction of CO from the last sucessful P step of 1130
my_analysis['P_bar'] = 1129. # P to start calculations off at next time. Chosen 1 bar below last successful step.

We'll run the next calcualtions at P step of 1 bar and using the "OHS" solve species function.

In [6]:
# start calculations from chosen P using the OHS solve_species option.
my_models = [['print status',"True"],['starting_P','set'],['solve_species','OHS']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# pressure step of 1 bar
degas2 = vf.calc_gassing(my_analysis,models=my_models,dp_step=1.)

Kīlauea_Dryend
T= 1220.0 P= 1711.9992252609893 2024-07-15 15:00:02.250410
1114.0 : Switching solve species from OHS to OCS (first time)
1114.0 : Switching solve species from OCS to OCH (second time)
solver failed, calculation aborted at P =  1114.0 2024-07-15 15:00:11.265213


It managed a few more bars - down to 1115 bar.

Same process to extract the last successful run...

In [7]:
my_analysis["xg_O2"] = degas2.loc[degas2['P_bar']==1115.0,'xgO2_mf']
my_analysis["xg_H2"] = degas2.loc[degas2['P_bar']==1115.0,'xgH2_mf']
my_analysis["xg_S2"] = degas2.loc[degas2['P_bar']==1115.0,'xgS2_mf']
my_analysis["xg_CO"] = degas2.loc[degas2['P_bar']==1115.0,'xgCO_mf']
my_analysis['P_bar'] = 1115.

I tried a few different options but turns out increasing the P step to 10 bars works for this example using OCH instead of OHS

In [8]:
# I also tell VolFe to start the calculation from 1099 bar given these initial guesses and to switch the solve species to OCH
my_models = [['print status',"True"],['starting_P','set'],['solve_species','OCH']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# pressure step back up to 10
degas3 = vf.calc_gassing(my_analysis,models=my_models,dp_step=10.)

Kīlauea_Dryend
T= 1220.0 P= 1711.9992252609893 2024-07-15 15:07:47.057374
1220.0 115.0 0.467583622103958  2024-07-15 15:08:02.634915
done 2024-07-15 15:08:04.532218


And we can amalgamate them into a single dataframe

In [9]:
degas1.reset_index(drop=True)
degas2.reset_index(drop=True)
degas3.reset_index(drop=True)
Kilauea_dry = pd.concat([degas1,degas2[1:],degas3[1:]])

### Kīlauea_Wetend ###

This one is much simpler and just runs all the way to 1 bar first time!

In [15]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Kīlauea_Wetend',
           'T_C': 1220., # Temperature in 'C
           'SiO2': 50.142, # wt%
           'TiO2': 2.257, # wt%
           'Al2O3': 12.224, # wt%
           'FeOT': 11.33, # wt%
           'MnO': 0.323, # wt%
           'MgO': 10.215, # wt%
           'CaO': 10.476, # wt%
           'Na2O': 1.754, # wt%
           'K2O': 0.398, # wt%
           'P2O5': 0.233, # wt%
           'H2O': 0.6, # wt%
           'CO2ppm': 800., # ppm
           'STppm': 1500., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.18}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])


In [16]:
# Choosing model options so that the print status works - just helps to see where the calculation is getting to
my_models = [['print status',"True"]]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

# running calculation
degas1 = vf.calc_gassing(my_analysis,models=my_models)

Kīlauea_Wetend
T= 1220.0 P= 1732.083512182475 2024-07-15 15:12:53.345361
1220.0 730.0 0.5809791230787278  2024-07-15 15:13:08.482866
done 2024-07-15 15:13:19.507007


### MORB ###

In [20]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'MORB',
           'T_C': 1100., # Temperature in 'C
           'SiO2': 47.4, # wt%
           'TiO2': 1.01, # wt%
           'Al2O3': 17.64, # wt%
           'FeOT': 7.98, # wt%
           'MnO': 0., # wt%
           'MgO': 7.63, # wt%
           'CaO': 12.44, # wt%
           'Na2O': 2.65, # wt%
           'K2O': 0.03, # wt%
           'P2O5':0.08, # wt%
           'H2O': 0.2, # wt%
           'CO2ppm': 1100., # ppm
           'STppm': 1400., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.1}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])


In [21]:
# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['print status',"True"]]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

degas1 = vf.calc_gassing(my_analysis,models=my_models)

MORB
T= 1100.0 P= 2193.9742708547615 2024-07-15 15:16:15.866159
1100.0 1190.0 -1.0001420572699367  2024-07-15 15:16:33.528527
910.0 : Switching solve species from OCH to OHS (first time)
860.0 : Switching solve species from OHS to OCS (first time)
860.0 : Switching solve species from OCS to OCH (second time)
solver failed, calculation aborted at P =  860.0 2024-07-15 15:16:46.528789


In [ ]:
my_analysis["xg_O2"] = degas1.loc[degas1['P_bar']==850.,'xgO2_mf']
my_analysis["xg_H2"] = degas1.loc[degas1['P_bar']==850.,'xgH2_mf']
my_analysis["xg_S2"] = degas1.loc[degas1['P_bar']==850.,'xgS2_mf']
my_analysis["xg_CO"] = degas1.loc[degas1['P_bar']==850.,'xgCO_mf']
my_analysis['P_bar'] = 1115.

OIB

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Fuego',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 51.46, # wt%
           'TiO2': 1.06, # wt%
           'Al2O3': 17.43, # wt%
           'FeOT': 9.42, # wt%
           'MnO': 0.19, # wt%
           'MgO': 3.78, # wt%
           'CaO': 7.99, # wt%
           'Na2O': 3.47, # wt%
           'K2O': 0.78, # wt%
           'P2O5': 0.24, # wt%
           'H2O': 2., # wt%
           'CO2ppm': 4000., # ppm
           'STppm': 4000., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.3}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

In [ ]:
Fuego = vf.calc_gassing(my_analysis)

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Fuego',
           'T_C': 1030., # Temperature in 'C
           'SiO2': 51.46, # wt%
           'TiO2': 1.06, # wt%
           'Al2O3': 17.43, # wt%
           'FeOT': 9.42, # wt%
           'MnO': 0.19, # wt%
           'MgO': 3.78, # wt%
           'CaO': 7.99, # wt%
           'Na2O': 3.47, # wt%
           'K2O': 0.78, # wt%
           'P2O5': 0.24, # wt%
           'H2O': 4.5, # wt%
           'CO2ppm': 3300., # ppm
           'STppm': 2650., # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.235}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['gassing_direction','regas'],['bulk_composition','melt+vapor_initialCO2'],['print status','True']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Fogo',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 42.4026, # wt%
           'TiO2': 3.254, # wt%
           'Al2O3': 11.174, # wt%
           'FeOT': 12.0352, # wt%
           'MnO': 0.1424, # wt%
           'MgO': 9.5482, # wt%
           'CaO': 13.3104, # wt%
           'Na2O': 3.3554, # wt%
           'K2O': 1.5734, # wt%
           'P2O5': 0.748, # wt%
           'H2O': 2.1054, # wt%
           'CO2ppm': 11515.4, # ppm
           'STppm': 4693.364, # ppm
           'Xppm': 0., # ppm
           'DNNO': 0.7}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Choose the options I want to change for the calculation - everything else will use the default options
my_models = [['gassing_direction','regas'],['bulk_composition','melt+vapor_initialCO2'],['print status','True']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)